# Splay木(平衡二分探索木)
## 概要
木が平衡に保たれるため、高速にアクセスできる二分探索木
### 計算量
追加、削除、探索: $O(logN)$

## 参考
- [【木マスター養成講座】7-1. Splay木ってなに〜？説明編【競プロかつっぱ】](https://www.youtube.com/watch?v=M6LcINhgXeM)
- [【木マスター養成講座】7-2. Splay木ってなに〜？実装編１【競プロかつっぱ】](https://www.youtube.com/watch?v=M6LcINhgXeM&t=0s)
- https://onlinejudge.u-aizu.ac.jp/solutions/problem/ITP2_1_A/review/4078592/catupper/C++14

## 方針

### クラス設計
Nodeに対応するclassを作る
```
class Node:
    val
    |- left, right, parent: 親と子のポインタ
    |- size: 部分木のサイズ
    |_ value: 値
    func
    |- rotate(): 回転
    |- splay(): meを上に上げる
    |_ update(): サイズの更新
```

### 回転

```
   pp-> o
        |
    p-> o
       / \
 me-> o   B
     / \
    A   C

       ↓

  pp-> o
       |
  me-> o
      / \
     A   o <-p
        / \
       C   B

```
- 全部で6つの代入
1. `pp.left = pp.right = me` ## ppがいるか確認
1. `me.parent = pp`
1. `me.right = p`
1. `p.parent = me`
1. `p.left = C`
1. `C.parent = p` ## Cがいるか確認

In [20]:
class Node:
    """スプレー木のノード
    Attributes:
        left, right, parent (Node): 親と子のノード
        size (int): 部分木のサイズ
        value: 値
    """
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        self.parent = None
        self.size = 1
    
    def __repr__(self):
        res = "Node(val:{}, left:{}, right:{}, parent:{})".format(
            self.val,
            self.left.val if self.left else None,
            self.right.val if self.right else None,
            self.parent.val if self.parent else None
        )
        return res

    def update(self):
        self.size = 1
        if node.left:
            self.size += self.left.size
        if self.right:
            self.size += self.right.size

    def state(self):
        if not self.parent:
            return 0
        if self.parent.left == self:
            return 1
        if self.parent.right == self:
            return -1
        return 0

In [21]:
class BSTree:
    def __init__(self):
        self.root = None
    
    def insert(self, node):
        """要素の挿入を行う
        Args:
            node (Node): 追加する要素
        """
        # 接点(parent)の探索
        parent = None
        ptr = self.root
        while ptr != None:
            parent = ptr
            ptr = parent.left if node.val < parent.val else parent.right
        
        # 要素の挿入
        node.parent = parent
        if parent == None:
            self.root = node
        elif node.val < parent.val:
            parent.left = node
        else:
            parent.right = node
    
    def delete(self, node):
        """要素の削除を行う

        Args:
            node (Node): 削除する要素
        """
        if node.left == None:
            self.transplant(node, node.right)
        elif node.right == None:
            self.transplant(node, node.left)
        else:
            y = self.search_min(node.right)

            # yがnodeの直下にない場合、yをnodeの真下に持ってくる必要がある
            if y.parent != node:
                # 一旦yを切り離す
                self.transplant(y, y.right)
                # yの右側にnodeの右側を貼り付ける
                y.right = node.right
                y.right.parent = y

            # yの左側にnodeの左側を貼り付ける
            self.transplant(node, y)
            y.left = node.left
            y.left.parent = y
    
    def transplant(self, u, v):
        """ノードの付け替えを行う

        Args:
            u (Node): 削除されるノード
            v (Node): 子となるノード
        """
        if u.parent == None:
            self.root = v
        elif u.parent.left == u:
            u.parent.left = v
        else:
            u.parent.right = v
        if v != None:
            v.parent = u.parent  # 親の更新
        
    @staticmethod
    def search_min(node) -> Node:
        """木の最小値の探索を行う

        Args:
            node (Node): 部分木のroot
        Returns:
            Node: 木の中で最小の値
        """
        while node.left:
            node = node.left
        
        return node
    
    def search(self, key) -> Node:
        """木の要素を探索する
        
        Args:
            key (Node): 探索する値
        Returns:
            Node: 見つかったノード
        """
        return self.partial_search(self.root, key)
    
    def partial_search(self, node, key) -> Node:
        """部分木の要素を探索する
        
        Args:
            root (Node): 部分木の根
            key : 探索する値
        Returns:
            Node: 見つかったノード
        """
        if node == None or node.val == key:
            return node
        elif key < node.val:
            return self.partial_search(node.left, key)
        else:
            return self.partial_search(node.right, key)
    
    def traverse(self, node=None):
        """木を巡回し、小さい順に値を返すジェネレータ

        Args:
            node=None (Node): 開始ノード
        Yields:
            Node: 見つかったノード
        """
        if node == None:
            node = self.root
        
        if node.left:
            yield from self.traverse(node.left)
        
        yield node

        if node.right:
            yield from self.traverse(node.right)
    
    def as_graph(self, format="png"):
        """グラフを可視化
        
        Args:
            format="png" (str): graphvizの出力形式
        Returns:
            graphviz.Digraph
        """

        graph = Digraph(format=format)
        fill = 0
        for node in self.traverse():
            v = str(node.val)
            graph.node(v)
        
        for node in self.traverse():
            v = str(node.val)
            if node.left:
                l = str(node.left.val)
                graph.edge(v, l)

            if node.right:
                r = str(node.right.val)
                graph.edge(v, r)
        
        return graph

In [22]:
class SplayNode(BSTree):
    def __init__(self):
        super().__init__()

    def search(self, key) -> Node:
        """木の要素を探索する
        
        Args:
            key (Node): 探索する値
        Returns:
            Node: 見つかったノード
        """
        return self.partial_search(self.root, key)
    
    def partial_search(self, node, key) -> Node:
        """部分木の要素を探索する
        
        Args:
            root (Node): 部分木の根
            key : 探索する値
        Returns:
            Node: 見つかったノード
        """
        if node == None or node.val == key:
            return node
        elif key < node.val:
            return self.partial_search(node.left, key)
        else:
            return self.partial_search(node.right, key)

    def insert(self, node):
        """要素の挿入を行う
        Args:
            node (Node): 追加する要素
        """
        # 接点(parent)の探索
        parent = None
        ptr = self.root
        while ptr != None:
            parent = ptr
            ptr = parent.left if node.val < parent.val else parent.right
        
        # 要素の挿入
        node.parent = parent
        if parent == None:
            self.root = node
        elif node.val < parent.val:
            parent.left = node
        else:
            parent.right = node
    
    def rotate(self, node):
        p = node.parent
        pp = p.parent

        if p.left == node:
            c = node.right
            node.right = p
            p.left = c
        else:
            c = node.left
            node.left = p
            p.right = c

        if pp and pp.left == p:
            pp.left = node
        if pp and pp.right == p:
            pp.right = node
        
        node.parent = pp
        p.parent = node
        if c:
            c.parent = p
        
        # 下から順にupdate
        p.update()
        node.update()

    def splay(self, node):
        while node.state() != 0:
            if node.parent.state() == 0:
                self.rotate(node)
            elif node.state() == node.parent.state():
                self.rotate(node.parent)
                self.rotate(node)
            else:
                self.rotate(node)
                self.rotate(node)


def get(ind, root):
    while True:
        lsize = root.left.size if root.left else 0
        if ind < lsize:
            root = root.left
        if ind == lsize:
            root.splay()
            return root
        if ind > lsize:
            root = root.right
            ind = ind - lsize - 1
    

In [23]:
class SplayNode:
    """スプレー木のノード
    Attributes:
        left, right, parent (SplayNode): 親と子のノード
        size (int): 部分木のサイズ
        value: 値
    """
    def __init__(self, value):
        self.left = self.right = None
        self.parent = None
        self.size = 1
        self.value = value
    
    def rotate(self):
        p = self.parent
        pp = p.parent

        if p.left == self:
            c = self.right
            self.right = p
            p.left = c
        else:
            c = self.left
            self.left = p
            p.right = c

        if pp and pp.left == p:
            pp.left = self
        if pp and pp.right == p:
            pp.right = self
        
        self.parent = pp
        p.parent = self
        if c:
            c.parent = p
        
        # 下から順にupdate
        p.update()
        self.update()

    def state(self):
        if not self.parent:
            return 0
        if self.parent.left == self:
            return 1
        if self.parent.right == self:
            return -1
        return 0

    def update(self):
        self.size = 1
        if self.left:
            self.size += self.left.size
        if self.right:
            self.size += self.right.size

    def splay(self):
        while self.state() != 0:
            if self.parent.state() == 0:
                self.rotate()
            elif self.state() == self.parent.state():
                self.parent.rotate()
                self.rotate()
            else:
                self.rotate()
                self.rotate()


def get(ind, root):
    while True:
        lsize = root.left.size if root.left else 0
        if ind < lsize:
            root = root.left
        if ind == lsize:
            root.splay()
            return root
        if ind > lsize:
            root = root.right
            ind = ind - lsize - 1
    

## リベンジ
参考
- http://www.nct9.ne.jp/m_hiroi/light/pyalgo20.html